In [1]:
import os
import pickle
import tensorflow as tf
import numpy as np
from network.test import TripletNet

# useful to inspect a checkpoint
from tensorflow.python.tools.inspect_checkpoint import print_tensors_in_checkpoint_file

In [2]:
log_dir = './log/sem2'
model_path = os.path.join(log_dir, 'ckpt')
epoch = 20//5 # checkpoint was made every 5 epochs

In [3]:
print_tensors_in_checkpoint_file('./log/sem2/ckpt/triplet-net-0', tensor_name='', all_tensors=False)

global_step (DT_INT64) []
triplet-net/_bn1/beta (DT_FLOAT) [32]
triplet-net/_bn1/beta/Momentum (DT_FLOAT) [32]
triplet-net/_bn1/gamma (DT_FLOAT) [32]
triplet-net/_bn1/gamma/Momentum (DT_FLOAT) [32]
triplet-net/_bn1/moving_mean (DT_FLOAT) [32]
triplet-net/_bn1/moving_variance (DT_FLOAT) [32]
triplet-net/_bn10/beta (DT_FLOAT) [256]
triplet-net/_bn10/beta/Momentum (DT_FLOAT) [256]
triplet-net/_bn10/gamma (DT_FLOAT) [256]
triplet-net/_bn10/gamma/Momentum (DT_FLOAT) [256]
triplet-net/_bn10/moving_mean (DT_FLOAT) [256]
triplet-net/_bn10/moving_variance (DT_FLOAT) [256]
triplet-net/_bn11/beta (DT_FLOAT) [512]
triplet-net/_bn11/beta/Momentum (DT_FLOAT) [512]
triplet-net/_bn11/gamma (DT_FLOAT) [512]
triplet-net/_bn11/gamma/Momentum (DT_FLOAT) [512]
triplet-net/_bn11/moving_mean (DT_FLOAT) [512]
triplet-net/_bn11/moving_variance (DT_FLOAT) [512]
triplet-net/_bn12/beta (DT_FLOAT) [1024]
triplet-net/_bn12/beta/Momentum (DT_FLOAT) [1024]
triplet-net/_bn12/gamma (DT_FLOAT) [1024]
triplet-net/_bn12/g

In [4]:
# load mean, std for training dataset
mean, std = pickle.load(open(os.path.join(log_dir, 'stats_sem.pkl'), 'rb'))
print('Mean: ', mean)
print('Std: ', std)

Mean:  0.30739194
Std:  0.21743536


In [5]:
# load network
net = TripletNet(model_path, epoch, mean=mean, std=std)

INFO:tensorflow:CNN: ftfy
INFO:tensorflow:Restoring parameters from ./log/sem2/ckpt/triplet-net-20


In [6]:
# basic: get features
images = np.random.random((1, 128, 128, 1)).astype(np.float32)
features = net.get_feature(images)
print('feature dim: ', features.shape)

feature dim:  (1, 128)


In [7]:
# basic: get features + get input
# input image doesn't need to be 128 x 128
# internally, it will resize the image into 128 x 128
images = np.random.random((1, 200, 200, 1)).astype(np.float32)
features = net.get_feature(images)
inIm = net.get_input(images)
print('input   dim: ', inIm.shape)
print('feature dim: ', features.shape)

input   dim:  (1, 128, 128, 1)
feature dim:  (1, 128)


In [8]:
# basic: get all tensors' outputs
# Note that tensor outputs are extracted after activation layer 
# (and pooling layer if it is followed after the activation layer)
# get_all_features returns list of dictionary where key is layer name and value is tensor output.
images = np.random.random((1, 200, 200, 1)).astype(np.float32)
outputs = net.get_all_features(images)

outputs = outputs[0]
for k, v in outputs.items():
    print("{}: {}".format(k, v.shape))


conv-01: (1, 128, 128, 32)
conv-02: (1, 64, 64, 64)
conv-03: (1, 64, 64, 128)
conv-04: (1, 64, 64, 64)
conv-05: (1, 32, 32, 128)
conv-06: (1, 32, 32, 256)
conv-07: (1, 32, 32, 128)
conv-08: (1, 16, 16, 256)
conv-09: (1, 16, 16, 512)
conv-10: (1, 16, 16, 256)
conv-11: (1, 8, 8, 512)
merged: (1, 8, 8, 3584)
conv-12: (1, 8, 8, 1024)
conv-13: (1, 8, 8, 512)
fc: (1, 128)
